In [6]:
%matplotlib

import numpy as np
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
from numpy import arange
import math
import time

fig, ax = plt.subplots()

pathx = []
pathy = []
hpathx = []
hpathy = []
opathx = []
opathy = []
opathx2 = []
opathy2 = []
opath_deg = []
patches = []

a_path, = plt.plot(pathx, pathy)
a_hpath, = plt.plot(hpathx, hpathy, 'x')
a_opath, = plt.plot(opathx, opathy, '-r')
a_opath2, = plt.plot(opathx, opathy, '-m*')
selected_circle = None

def add_path(x, y):
    pathx.append(x)
    pathy.append(y)

def add_helper_point(x, y):
    hpathx.append(x)
    hpathy.append(y)

def add_opti_path(x, y):
    opathx.append(x)
    opathy.append(y)

def add_opti2_path(x, y):
    opathx2.append(x)
    opathy2.append(y)


circles = []

circles.append((0,0,0.4))
circles.append((1,7,2))
circles.append((9,9,2))
circles.append((6,2,1))
circles.append((10,0,2))
circles.append((13,7,0.4))

opath_deg = [0] * len(circles)

def recalc_optimal2():
    global tdist
    global tdist2
    
    global opathx2
    global opathy2
    
    opathx2 = []
    opathy2 = []
    
    add_opti2_path(*circles[0][:2])
    for i in range(len(opath_deg) - 2):
        i += 1
        if opath_deg[i] is not None:
            rad = math.radians(opath_deg[i])
            cx, cy, r = circles[i]
            x = cx + math.cos(rad) * r
            y = cy + math.sin(rad) * r
            add_opti2_path(x, y)
    add_opti2_path(*circles[-1][:2])

    tdist = 0
    tdist2 = 0

    for i in range(len(opathx) - 1):
        tdist += math.sqrt(pow(opathx[i] - opathx[i + 1], 2) + pow(opathy[i] - opathy[i + 1], 2))

    for i in range(len(opathx2) - 1):
        tdist2 += math.sqrt(pow(opathx2[i] - opathx2[i + 1], 2) + pow(opathy2[i] - opathy2[i + 1], 2))

def redraw(selected_circle = None, calc_circles = None):
    if (calc_circles == None):
        calc_circles = list(circles)
    
    global pathx
    global pathy
    global hpathx
    global hpathy
    global opathx
    global opathy
    global opathx2
    global opathy2
    global opath_deg
    global patches
    
    pathx = []
    pathy = []
    hpathx = []
    hpathy = []
    opathx = []
    opathy = []

    
    add_path(*circles[0][:2])
    for x, y, r in circles:
        add_path(x, y)


    #SOLVER
    def solve_point2circle(x, y, r, cx, cy, cr):
        v_x = x - cx
        v_y = y - cy
        magV = math.sqrt(pow(v_x, 2) + pow(v_y, 2))
        r_x = cx + v_x / magV * cr
        r_y = cy + v_y / magV * cr    

        v_x = ((v_x / magV))
        v_y = ((v_y / magV))    
        return r_x, r_y, v_x, v_y

    def solve_vector2circle(v_x, v_y, cx, cy, r):
        magV = math.sqrt(pow(v_x, 2) + pow(v_y, 2))
        r_x = cx + v_x / magV * r
        r_y = cy + v_y / magV * r    

        return r_x, r_y

    def get_opti_point(pindex, index, nindex):
        x, y, r = calc_circles[index]
        if pindex < 0 or nindex >= len(calc_circles):
            return x, y
        
        prev_x, prev_y, prev_r = calc_circles[pindex]
        next_x, nexy_y, next_r = calc_circles[nindex]

        rx, ry, vx1, vy1 = solve_point2circle(prev_x, prev_y, prev_r, x, y, r)
        add_helper_point(rx, ry)
        rx, ry, vx2, vy2 = solve_point2circle(next_x, nexy_y, next_r, x, y, r)
        add_helper_point(rx, ry)

        
        vx = x + vx1 + vx2
        vy = y + vy1 + vy2

#         if index == selected_circle:
#             add_opti2_path(x, y)
#             add_opti2_path(x + vx1, y + vy1)
#             add_opti2_path(vx, vy)
#             add_opti2_path(x + vx2, y + vy2)
#             add_opti2_path(x, y)
#             add_opti2_path(vx, vy)
#             add_opti2_path(None, None)


        rx, ry, __, __ = solve_point2circle(vx, vy, 0, x, y, r)
        add_helper_point(rx, ry)
        return rx, ry
        

    add_opti_path(*calc_circles[0][:2])

    for i in range(len(calc_circles) - 2):
        i = i + 1
        rx, ry = get_opti_point(i - 1, i, i + 1)
        add_opti_path(rx, ry)

    add_opti_path(*calc_circles[-1][:2])



    for p in patches:
        p.remove()
    
    patches = []
    p = []
    i = 0
    for x, y, r in circles:
        if i == selected_circle:
            circle = Circle((x, y), r, alpha=0.6, ls='-', lw=1)
        else:
            circle = Circle((x, y), r, alpha=0.2, ls='-', lw=1)
            
        patches.append(ax.add_patch(circle))
        i += 1


    recalc_optimal2()

    
    fig.canvas.set_window_title("tot:%0.3f,%0.3f,%0.3f " % (tdist, tdist2, tdist2 - tdist))
    
    
    a_path.set_data(pathx, pathy)
    a_hpath.set_data(hpathx, hpathy)
    a_opath.set_data(opathx, opathy)
    a_opath2.set_data(opathx2, opathy2)
    fig.canvas.draw_idle()
#    fig.canvas.draw()

itter = [0]

def ev(event):
    x = event.xdata
    y = event.ydata
    if x is None or x is None:
        return
    
    global selected_circle
    global itter

    selected_circle = None
    for i in range(len(circles)):
        cx, cy, r = circles[i]
        if math.sqrt(pow(x - cx, 2) + pow(y - cy, 2)) <= r:
            selected_circle = i
    
    if event.button == 1 and selected_circle is not None:        # right mouse button
        cx, cy, r = circles[selected_circle]
        circles[selected_circle] = x, y, r
        redraw(selected_circle)
        itter = [0]
        
    if event.button == 2 and selected_circle is not None:        # middle mouse button
        
        start_time = time.process_time()
        while len(itter) < 5:
            improve = False
            for i in range(len(circles) - 2):
                i += 1
                min_dest = 1000000000000
                min_angle = None
                min_a = 100000000000000000

                last_angle = opath_deg[i]

                global tdist2

                if len(itter) == 1:
                    start = 0
                    end = 360
                    step = 10

                if len(itter) == 2:
                    start = last_angle - 18
                    end = last_angle + 18
                    step = 1

                if len(itter) == 3:
                    start = last_angle - 0.5
                    end = last_angle + 0.5
                    step = 0.1     
                    
                if len(itter) == 4:
                    start = last_angle - 0.05
                    end = last_angle + 0.05
                    step = 0.01                

                for angle in arange(start, end, step):
                    opath_deg[i] = angle
                    recalc_optimal2()

                    a = math.sqrt(pow(opathx2[i] - opathx2[i - 1], 2) + pow(opathy2[i] - opathy2[i - 1], 2))
                    b = math.sqrt(pow(opathx2[i] - opathx2[i + 1], 2) + pow(opathy2[i] - opathy2[i + 1], 2))
                    c = a + b
                    
                    if min_dest > c:
                        min_dest = c
                        min_a = a
                        min_angle = angle


                if last_angle is None:
                    opath_deg[i] = min_angle  
                    improve = True
                    continue

                if abs(last_angle - min_angle) > step:
                    print(last_angle, min_angle)
                    opath_deg[i] = min_angle    
                    improve = True
                else:
                    opath_deg[i] = last_angle 

            itter[-1] += 1
            if not improve:
                itter.append(0)

            total = "%s elapsed %0.3fs" % (str(itter), time.process_time() - start_time)
            ax.set_title(total)

            redraw(None)      
            
            

       
        
def sc(event):
    global selected_circle
    
    if event.button == 'up' and selected_circle is not None:      
        if opath_deg[selected_circle] == None:
            opath_deg[selected_circle] = 0
        opath_deg[selected_circle] = (opath_deg[selected_circle] + 1 % 360)

        redraw(selected_circle)

    if event.button == 'down' and selected_circle is not None:      
        if opath_deg[selected_circle] == None:
            opath_deg[selected_circle] = 0
        opath_deg[selected_circle] = ((opath_deg[selected_circle] - 1 + 360)  % 360)

        redraw(selected_circle)

ax.set_xlim(-2, 16)
ax.set_ylim(-2, 16)

redraw(None)        


a_opath.figure.canvas.mpl_connect('motion_notify_event', ev)
a_opath.figure.canvas.mpl_connect('button_press_event', ev)
fig.canvas.mpl_connect('scroll_event', sc)

plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
plt.axis('equal')
plt.show()

Using matplotlib backend: TkAgg
0 300
0 340
0 310
0 100
340 170
310 190
300 295
190 194
100 105
170 172
194 200
295 294.5
172 172.3
200 200.3
105 105.4
294.5 294.1
172.3 172.6
200.3 200.7
105.4 105.8
200.7 201.1
201.1 201.5
105.8 106.0
201.5 201.7
294.1 294.05
172.6 172.65
201.7 201.72
294.05 294.0
172.65 172.7
201.72 201.77
294.0 293.95
172.7 172.75
201.77 201.82
293.95 293.9
172.75 172.8
201.82 201.87
293.9 293.88
172.8 172.84
201.87 201.92
106.0 106.01
293.88 293.86
172.84 172.86
201.92 201.95
293.86 293.84
172.86 320
201.95 10
106.01 120
320 170
10 110
170 0
293.84 299.84
110 97
299.84 300.14
0 0.4
97 96.5
120 119.5
0.4 0.8
96.5 96.0
300.14 300.24
0.8 1.0
96.0 95.5
119.5 119.4
95.5 95.0
95.0 94.5
94.5 94.0
1.0 1.2
94.0 93.5
300.24 300.34
93.5 93.0
93.0 92.5
92.5 92.0
92.0 91.5
1.2 1.4
91.5 91.0
91.0 90.5
90.5 90.0
90.0 89.5
89.5 89.0
119.4 119.3
89.0 88.5
1.4 1.6
88.5 88.0
88.0 87.5
87.5 87.0
87.0 86.5
86.5 86.0
1.6 1.7
86.0 85.5
300.34 300.44
85.5 85.0
85.0 84.5
300.44 300.39
84.5